In [1]:
from datasets import load_dataset

ds = load_dataset("nvidia/HelpSteer2")

README.md:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.jsonl.gz:   0%|          | 0.00/582k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1038 [00:00<?, ? examples/s]

In [2]:
"""import pandas as pd

df = ds['train'].to_pandas()

group = df.groupby('prompt')

criterion = ['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity']
res = []
for n, g in group:
    dct = {}
    v, w = g.iloc[0], g.iloc[1]
    score = 0
    for c in criterion:
        # 0 if v loses
        # 1 if tie
        # 2 if v wins
        if v[c] > w[c]:
            score += 2
        elif v[c] < w[c]:
            pass
        else:
            score += 1
        if score == 0:
            score += 1
    dct['prompt1'] = v['prompt']
    dct['prompt2'] = w['prompt']
    dct['label'] = score
    res.append(dct)

from datasets import Dataset
dataset = Dataset.from_list(res)"""

"import pandas as pd\n\ndf = ds['train'].to_pandas()\n\ngroup = df.groupby('prompt')\n\ncriterion = ['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity']\nres = []\nfor n, g in group:\n    dct = {}\n    v, w = g.iloc[0], g.iloc[1]\n    score = 0\n    for c in criterion:\n        # 0 if v loses\n        # 1 if tie\n        # 2 if v wins\n        if v[c] > w[c]:\n            score += 2\n        elif v[c] < w[c]:\n            pass\n        else:\n            score += 1\n        if score == 0:\n            score += 1\n    dct['prompt1'] = v['prompt']\n    dct['prompt2'] = w['prompt']\n    dct['label'] = score\n    res.append(dct)\n\nfrom datasets import Dataset\ndataset = Dataset.from_list(res)"

In [3]:
df = ds['train'].to_pandas()


In [4]:
df['inp'] = df['prompt']+'\n'+df['response']

In [5]:
df['score'] = df[['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity']].sum(1)

In [6]:
bins = range(0, 22, 2)  # Bins from 0 to 20 (inclusive), with step 2
labels = range(0, 10)   # Labels 1 to 10

In [7]:
import pandas as pd

In [8]:
df['label'] = pd.cut(df['score'], bins=bins, labels=labels, right=True, include_lowest=True)

In [9]:
dataset = []
for i, row in df.iterrows():
    print(row['inp'])
    break

c#
C# is a high-level, object-oriented programming language developed by Microsoft as part of its .NET initiative. It was created as a modern alternative to Java and supports a variety of programming paradigms, including imperative, functional, and event-driven. C# is primarily used for Windows application development, but it can also be used for web, mobile, and game development. The language is designed to be safe, secure, and efficient, and it provides developers with a rich set of libraries and tools for building robust and scalable applications. C# is also widely used in the game development industry, particularly in the development of games for the Xbox 360 and Xbox One consoles.


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

PATH_SFT = '/kaggle/input/sft-opt/sft_opt/'

tokenizer = AutoTokenizer.from_pretrained(PATH_SFT, local_files_only=True, padding_side='left')

reward_model = AutoModelForSequenceClassification.from_pretrained(
    PATH_SFT, local_files_only=True,
    num_labels=10,  # 10 classes
    problem_type="single_label_classification"  # Default behavior for single-label classification
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/sft-opt/sft_opt/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
dataset = []
for i, row in df.iterrows():
    dataset.append({'text': row['inp'], 'label': row['label']})

In [12]:
dataset[0]

{'text': 'c#\nC# is a high-level, object-oriented programming language developed by Microsoft as part of its .NET initiative. It was created as a modern alternative to Java and supports a variety of programming paradigms, including imperative, functional, and event-driven. C# is primarily used for Windows application development, but it can also be used for web, mobile, and game development. The language is designed to be safe, secure, and efficient, and it provides developers with a rich set of libraries and tools for building robust and scalable applications. C# is also widely used in the game development industry, particularly in the development of games for the Xbox 360 and Xbox One consoles.',
 'label': 6}

In [13]:
def preprocess(example):
    encoding = tokenizer(
        example["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=256,  # Adjust max_length as needed
    )
    encoding["labels"] = example["label"]  # Add label to the encoding
    return encoding

In [14]:
from datasets import Dataset

In [15]:
raw_dataset = Dataset.from_list(dataset)
tokenized_dataset = raw_dataset.map(preprocess, batched=False)

Map:   0%|          | 0/20324 [00:00<?, ? examples/s]

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    fp16=True,
    save_strategy="no", save_total_limit=1, logging_steps=50,
    
)

trainer = Trainer( 
    model=reward_model,
    args=training_args,
    train_dataset=tokenized_dataset
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
50,2.421600
100,1.954000
150,1.837300
200,1.805500
250,1.859500
300,1.803200
350,1.752400
400,1.752900
450,1.707800
500,1.720100


TrainOutput(global_step=2541, training_loss=1.7364972027114693, metrics={'train_runtime': 1745.0309, 'train_samples_per_second': 11.647, 'train_steps_per_second': 1.456, 'total_flos': 9470286451703808.0, 'train_loss': 1.7364972027114693, 'epoch': 1.0})

In [18]:
trainer.save_model('RM_opt_2')